In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc
from pyspark.sql.functions import col
from pyspark.sql.functions import lower, explode, split
from pyspark.sql.functions import avg
 
spark_session = SparkSession.builder\
        .master("spark://192.168.2.92:7077")\
        .appName("ubuntu@group17-master")\
        .getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
spark_context = spark_session.sparkContext

spark_context.setLogLevel("ERROR")

In [3]:
spark_context = spark_session.sparkContext

In [4]:
#df = spark_session.read.json("hdfs://192.168.2.92:9000/user/group17/sample_data.json")
df1 = spark_session.read.json("hdfs://192.168.2.92:9000/user/group17/RC_2020-01-29.json")


In [5]:
df1 = df1.drop('author_flair_background_color', 'uthor_flair_css_class',
               'author_flair_richtext','author_flair_template_id','author_flair_text',
               'author_flair_text_color','author_flair_type','author_patreon_flair',
               'gilded','distinguished','locked'
               'can_gild','gildings', 'media_metadata','quarantined')

In [5]:
df1.head()

Row(all_awardings=[], associated_award=None, author='WestCoastSpiderman', author_cakeday=None, author_created_utc=None, author_flair_background_color=None, author_flair_css_class='packers', author_flair_richtext=[Row(a=None, e='text', t='Packers', u=None)], author_flair_template_id='3bbca72c-3b9e-11ea-9224-0e8e77fdb897', author_flair_text='Packers', author_flair_text_color='dark', author_flair_type='richtext', author_fullname='t2_4jocp94p', author_patreon_flair=False, author_premium=False, awarders=[], body='Got my apartment in Utah 😎', can_gild=True, can_mod_post=False, collapsed=False, collapsed_because_crowd_control=None, collapsed_reason=None, controversiality=0, created_utc=1580256000, distinguished=None, edited='false', gilded=0, gildings=Row(gid_1=None, gid_2=None, gid_3=None), id='ffv5a10', is_submitter=False, link_id='t3_ev83qk', locked=False, media_metadata=None, no_follow=False, parent_id='t3_ev83qk', permalink='/r/nfl/comments/ev83qk/talko_tuesday/ffv5a10/', quarantined=Fal

In [ ]:
df1.take(5)

In [6]:
df1.describe()

DataFrame[summary: string, associated_award: string, author: string, author_created_utc: string, author_flair_background_color: string, author_flair_css_class: string, author_flair_template_id: string, author_flair_text: string, author_flair_text_color: string, author_flair_type: string, author_fullname: string, body: string, collapsed_because_crowd_control: string, collapsed_reason: string, controversiality: string, created_utc: string, distinguished: string, edited: string, gilded: string, id: string, link_id: string, parent_id: string, permalink: string, removal_reason: string, retrieved_on: string, score: string, subreddit: string, subreddit_id: string, subreddit_name_prefixed: string, subreddit_type: string, total_awards_received: string]

In [7]:
comments = df1.groupBy("subreddit").count()

top_subreddits = comments.orderBy(desc("count")).limit(10)
print("Top 10 most commented subreddits")
top_subreddits.show()

Top 10 most commented subreddits


+----------------+------+
|       subreddit| count|
+----------------+------+
|       AskReddit|264521|
|        politics| 99374|
|           memes| 70071|
|   AmItheAsshole| 43449|
|       teenagers| 39997|
|       worldnews| 36778|
|       dankmemes| 36667|
|unpopularopinion| 32387|
|             nba| 30070|
|      The_Donald| 28331|
+----------------+------+



In [21]:
score = df1.select("subreddit", "score")

score = score.groupBy("subreddit")
score = score.agg(avg("score").alias("avg_score_comment"))

topten_subreddits = ["AskReddit", "politics", "memes", "AmItheAsshole",
                     "teenagers", "worldnews", "dankmemes", 
                     "unpopularopinion", "nba", "The_Donald"]

score = score.filter(score.subreddit.isin(topten_subreddits))
highest_scores = score.groupBy("subreddit").agg({"avg_score_comment": "first"})
highest_scores = highest_scores.orderBy(desc("first(avg_score_comment)"))
highest_scores.show()

+----------------+------------------------+
|       subreddit|first(avg_score_comment)|
+----------------+------------------------+
|             nba|      21.898636514798802|
|   AmItheAsshole|      20.841699463739097|
|       AskReddit|       16.33772743940935|
|       worldnews|      15.445103050736854|
|        politics|      10.098416084690161|
|       dankmemes|        9.34930591540077|
|      The_Donald|       8.127987010694998|
|           memes|       6.723922878223516|
|       teenagers|       4.875340650548791|
|unpopularopinion|      3.4307283786704543|
+----------------+------------------------+



In [13]:
spark_session.stop()